In [1]:
!nvidia-smi

Wed Nov 26 11:49:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="lBA8e9Ed1FdiTmZbCRC1")
project = rf.workspace("my-workspace-xq9bt").project("football_players_and_ball-fqw10")
version = project.version(5)
dataset = version.download("yolov12")

loading Roboflow workspace...
loading Roboflow project...


In [9]:
dataset.location

'/content/drive/MyDrive/ball_detection/datasets/ball'

In [4]:
!mv {dataset.location} /content/drive/MyDrive/ball_detection/datasets/

In [8]:
dataset.location = '/content/drive/MyDrive/ball_detection/datasets/ball'

In [10]:
import os
import shutil

# ----------------------------
# CONFIG
# ----------------------------
dataset_root = dataset.location               # Folder containing train/ valid/ test/
remove_cls = 1                    # <-- class ID to remove (0=ball, 1=player)
output_folder = f"/content/drive/MyDrive/ball_detection/datasets/filtered_remove_{remove_cls}"
# ----------------------------

def copy_dataset(src, dst):
    """Copy entire dataset directory except data.yaml."""
    if os.path.exists(dst):
        shutil.rmtree(dst)
    shutil.copytree(src, dst, ignore=shutil.ignore_patterns("data.yaml"))
    print(f"Copied dataset to: {dst}")

def filter_labels(folder, remove_cls):
    """Remove lines with remove_cls from YOLO label files."""
    labels_dir = os.path.join(folder, "labels")

    if not os.path.exists(labels_dir):
        print(f"Labels directory not found: {labels_dir}")
        return

    for fname in os.listdir(labels_dir):
        if not fname.endswith(".txt"):
            continue

        path = os.path.join(labels_dir, fname)

        with open(path, "r") as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if not parts:
                continue

            cls_id = int(parts[0])

            if cls_id == remove_cls:
                # skip this object
                continue

            new_lines.append(line)

        # Always write the file, even if empty
        with open(path, "w") as f:
            f.writelines(new_lines)

    print(f"Filtered labels in: {labels_dir}")

# ----------------------------
# RUN PROCESS
# ----------------------------
copy_dataset(dataset_root, output_folder)

for split in ["train", "valid", "test"]:
    split_dir = os.path.join(output_folder, split)
    if os.path.exists(split_dir):
        filter_labels(split_dir, remove_cls)

print("DONE ✔ Dataset filtered successfully.")


Copied dataset to: /content/drive/MyDrive/ball_detection/datasets/filtered_remove_1
Filtered labels in: /content/drive/MyDrive/ball_detection/datasets/filtered_remove_1/train/labels
Filtered labels in: /content/drive/MyDrive/ball_detection/datasets/filtered_remove_1/valid/labels
Filtered labels in: /content/drive/MyDrive/ball_detection/datasets/filtered_remove_1/test/labels
DONE ✔ Dataset filtered successfully.
